# Superface Agent Hub - Open AI Function Calling Example
In this notebook we demonstrate how to use the Superface Agent Hub to connect your OpenAI powered agent to external tools and APIs in a way that allows for both personal and third-party use.

In [1]:
%pip install openai --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.11.0 requires anyio<4,>=3.1.0, but you have anyio 4.3.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import openai
import json
import requests as r
from openai import OpenAI
from IPython.display import display, Markdown

# Set a random number of your choice, but don't change it
# once you have run the notebook, otherwise you will create another user.
SUPERFACE_USER_ID_CONSTANT = 

# Use the number to create a unique ID
SUPERFACE_USER_ID = "sfoaidemo|" + str(SUPERFACE_USER_ID_CONSTANT)

# Default URL for Superface
SUPERFACE_BASE_URL = "https://pod.superface.ai/api/hub"

# Set the Superface authentication token
SUPERFACE_AUTH_TOKEN="<your-superface-auth-token>"

# Set the OpenAI API Key
OPENAI_API_KEY="<your-open-api-key>"


# OpenAI Config
client = OpenAI(api_key=OPENAI_API_KEY)
GPT_MODEL = "gpt-4-turbo-preview"
INIT_INSTRUCTIONS = """
You are a helpful assistant.
Respond to the following prompt by using function_call and then summarize actions.
Ask for clarification if a user request is ambiguous.
Display the agent_hint from the response to the user if it is present.
"""

In [3]:
# Helper function to return the tool function descriptors
def get_superface_tools():
  headers = {"Authorization": "Bearer "+ SUPERFACE_AUTH_TOKEN}
  tools = r.get(SUPERFACE_BASE_URL + "/fd", headers=headers)
  return tools.json()

# Helper function to perform the action for all the functions.
# This is the only API call required regardless of what the function is.
def perform_action(tool_name=None, tool_body=None):
  headers = {"Authorization": "Bearer "+ SUPERFACE_AUTH_TOKEN, "x-superface-user-id": SUPERFACE_USER_ID}
  perform = r.post(SUPERFACE_BASE_URL + "/perform/" + tool_name, headers=headers, json=tool_body)
  return json.dumps(perform.json())

# Helper function for calling the OpenAI Chat Completions API
def perform_chat_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
  try:
    response = client.chat.completions.create(
      model=model,
      messages=messages,
      tools=tools,
      tool_choice=tool_choice,
    )
    return response
  except Exception as e:
    print("Unable to generate ChatCompletion response")
    print(f"Exception: {e}")
    return e

In [4]:
# User prompt - The weather tool requires no authentication
prompt = "What is the weather in Prague?"


In [5]:
messages = []
messages.append({"role": "system", "content": INIT_INSTRUCTIONS})


messages.append({
  "role": "user",
  "content": prompt
})

chat_response = perform_chat_request(
  messages, tools=get_superface_tools()
)

assistant_message = chat_response.choices[0].message
messages.append(assistant_message)

#assistant_message

tool_calls = assistant_message.tool_calls

if (assistant_message.tool_calls):
  # Assistant wants to call a tool, run it

  for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    function_response = perform_action(function_name, function_args)
    #print(function_response)

    messages.append(
      {
        "tool_call_id": tool_call.id,
        "role": "tool",
        "name": function_name,
        "content": function_response,
      }
    )

    second_chat_response = perform_chat_request(messages, tools=get_superface_tools())
    print(second_chat_response)

    if second_chat_response.choices[0].message.content:
      display(Markdown(second_chat_response.choices[0].message.content))

ChatCompletion(id='chatcmpl-99u9jr7AmC358uN6FtFYumvuf720A', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current weather in Prague, Czech Republic is partly cloudy with a temperature of 13°C, and it feels like 13°C.', role='assistant', function_call=None, tool_calls=None))], created=1712147843, model='gpt-4-0125-preview', object='chat.completion', system_fingerprint='fp_a7daf7c51e', usage=CompletionUsage(completion_tokens=28, prompt_tokens=2314, total_tokens=2342))


The current weather in Prague, Czech Republic is partly cloudy with a temperature of 13°C, and it feels like 13°C.